<a href="https://colab.research.google.com/github/Hoangthang017/CS336.L11/blob/master/Do_an_cuoi_ki/main_search_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **clone github**

In [ ]:
# clone github
!git clone https://github.com/Hoangthang017/CS336.L11.git
!pip install underthesea

# **import thư viện**

In [2]:
from glob import glob
import pandas as pd
import pickle
import re
import string
import numpy as np
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
sys.path.insert(1, '/content/CS336.L11/Do_an_cuoi_ki/import_file')
import create_tfidf

# **load đường dẫn file**

In [3]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/bong_da_v1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/")
# load dataset
files_path = glob("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/*/*.txt")

In [4]:
# print(files_path)

# **load tfidf file **

In [5]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/")
# load data from gg drive 
df_tfidf = pd.read_csv('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.csv', index_col=0)

In [6]:
# print(df_tfidf)

# **load model vectorizer** 

In [7]:
# load vectorizer
vectorizer = pickle.load(open('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/vectorizer.pk', 'rb'))

# **hàm chính** 

## *Nhập câu truy vấn và thực hiện truy vấn*

In [10]:
# câu truy vấn
query = 'barcelona'
# tiền xử lí câu truy vấn
query = create_tfidf.text_preprocess(query)
# gọi hàm truy vấn
sim_sorted = create_tfidf.get_similar_articles(query,df_tfidf,vectorizer)

## *Xuất kết quả truy vấn*

In [11]:
# số lượng bài viết tìm được
rank = 10
now = 0

# in kết quả truy vấn được
for k, v in sim_sorted:
  print("Độ tương đồng: ", v)
  # print("Tiêu đề: ", files_name[k])
  print(files_path[k])
  now += 1
  if (now == rank):
    break

Độ tương đồng:  0.2173394309208078
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Tây Ban Nha/Guardiola được nhắm đưa về để _cứu_ Barca.txt
Độ tương đồng:  0.20287261288742134
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Tây Ban Nha/Với Real, El Clasico  là một... _phiên tòa_.txt
Độ tương đồng:  0.17597054697633088
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Tây Ban Nha/Barca khủng hoảng tài chính do Messi nhiều hơn do Covid-19.txt
Độ tương đồng:  0.15743961046116814
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Champions League/Koeman vs Pirlo_ Chỉ là bóng đá thôi mà!.txt
Độ tương đồng:  0.14588722669930643
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Tây Ban Nha/Zidane chưa từng thua tại Nou Camp ở El Clasico.txt
Độ tương đồng:  0.12673571911561704
/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Tây Ban Nha/_Messi đang rất khó chịu tại Barca_.txt
Độ tương đồng:  0.11290587867764058
/content/CS336.L11/Do_an_cuoi_